# Access rancher info 

  pip install git+https://github.com/rancher/client-python.git@master
  
get credentials from Rancher click on avatar select API Keys 
Create a new API key
put bearer_token in secrets.json

### secrets.son file should look like 
{
    "access_key": "token-fq..",
    "secret_key": "ihg9x...",
    "bearer_token": "token-fq....ih...."
}

In [115]:
import os
import json
from time import time
from IPython.display import HTML as html_print


filename=os.path.join('secrets.json')
secrets={}
with open(filename, mode='r') as f:
    secrets=json.loads(f.read()) 
    


In [51]:
import rancher
url='https://rancher.cp.lsst.org/v3/'

# can not be cluster scoped
client = rancher.Client(url=url,token=secrets['bearer_token'])


In [102]:
# this gets the info we need
nodes = client.list('node')['data']


In [126]:
# this builds a simple traffic like status
status = -1
expect = 10
cluster = 'yagan'
stat = [['Bad','red'],['Fair','orange'],['Fine','green']]

nodes = [ n 
          for n in nodes if n['hostname'].startswith(cluster)
        ] 
nodes = sorted(nodes, key= lambda n: n.hostname)

#print(f"Have {len(nodes)} nodes of 10 expected.")
if len(nodes) == expect:
    status=0

times = {}
readyc = 0    

for i,n in enumerate(nodes):
#    print(f"{i} {n['hostname']} {n['conditions'][3]['lastHeartbeatTime']} {n['conditions'][3]['type']} {n['etcd']} ")
    times[n['hostname']] = [n['conditions'][3]['lastHeartbeatTimeTS'], n['conditions'][3]['lastHeartbeatTime']]
    if 'Ready' in {n['conditions'][3]['type']}:
        readyc = readyc + 1

if (readyc == expect):
    status = status + 1
    
times = sorted(times.items(), key=lambda t: t[1])                 
now = time()
howlong = (int)((now - times[0][1][0]/1000)/60)

if howlong < 10 :
   status = status + 1

print (f"Oldest heartbeat: {times[0][0]} {times[0][1][1]} is {howlong}s old {stat[status]}")

html_print(f"<text style=color:{stat[status][1]}>{cluster} if {stat[status][0]}</text>")


Oldest heartbeat: yagan03 2022-12-19T17:35:28Z is 33s old ['Fair', 'orange']
